# c) Ngonye Falls Flow Analysis

Load the synthetic historic daily flow series for Ngonye and produce various summary statistics for later presentation.

## Inputs

| Data                       | Source                                        | Description                                 |
|----------------------------|-----------------------------------------------|---------------------------------------------|
| ngonye_synthetic.csv  | Notebook: b_synthetic_flow_ngonye |Synthetic daily flow series for Ngonye  Falls 1924/25 - 2016/17  |
| selected_years.csv | Mott MacDonald - Ngonye Falls Hydropower Project - 2018 Feasibility Study Update - Final Report Version D | List of representative selected years |


## Outputs
| File                           | Description                                 |
|--------------------------------|---------------------------------------------|
| ngonye_flow_daily.csv          | Daily flow data  |
| ngonye_flow_monthly.csv        | Flow summaries by month  |
| ngonye_flow_yearly.csv         | Flow summaries by year  |
| ngonye_flow_calmonthly.csv     | Flow summaries by calendar month |
| ngonye_flow_selected_years.csv | Flow summaries for selected representative years  |



## Parameters

In [110]:
input_data='./input_data/'
output_data='./output_data/'

## Libraries

In [111]:
import numpy as np
import pandas as pd

## Load the Daily Data

In [112]:
daily = pd.read_csv(output_data + "ngonye_synthetic.csv")
daily.tail(4)

,Date,LaggedDate,VicFalls,Conversion,Flow,Exceedance
33964,2017-09-27,2017-10-08,204.0,0.995184,203.017496,0.950
33965,2017-09-28,2017-10-09,204.0,0.995184,203.017496,0.950
33966,2017-09-29,2017-10-10,204.0,0.995184,203.017496,0.950
33967,2017-09-30,2017-10-11,201.0,0.986217,198.229576,0.958


Index by date and add some other columns for later use. 

Add a column for *WaterYear* which starts on 1st October and runs to 31st September the following year.

In [113]:
daily['Date']=pd.to_datetime(daily['Date'],format="%Y-%m-%d")#"%d/%m/%Y")
daily=daily.set_index(pd.DatetimeIndex(daily['Date']))


In [114]:
daily['Year']=daily.index.year
daily['Month']=daily.index.month
daily['Day']=daily.index.day
daily['MonthId']=daily['Year']+daily['Month']/100
daily['WaterYear']=daily.apply((lambda x: (x['Year'] if x['Month']>=10 else x['Year']-1)),axis=1)
daily['WaterMonth']=daily.apply((lambda x: (x['Month']-9 if x['Month']>=10 else x['Month']+3)),axis=1)
daily['WaterDay']=daily.apply(lambda x: (x['Date']-pd.Timestamp(x['WaterYear'], 10, 1)).days+1,axis=1)
daily['WaterWeek']=np.floor((daily['WaterDay']-1)/7)+1
daily['Volume']=daily['Flow']*60*60*24/(1000*1000*1000)
daily=daily.astype({'WaterWeek': 'int32'})
daily=daily.drop('Date',axis=1)
daily.head(8)

,LaggedDate,VicFalls,Conversion,Flow,Exceedance,Year,Month,Day,MonthId,WaterYear,WaterMonth,WaterDay,WaterWeek,Volume
Date,,,,,,,,,,,,,,
1924-10-01,1924-10-12,100.0,1.111331,111.133124,0.999,1924,10,1,1924.1,1924,1,1,1,0.009602
1924-10-02,1924-10-13,100.0,1.111331,111.133124,0.999,1924,10,2,1924.1,1924,1,2,1,0.009602
1924-10-03,1924-10-14,100.0,1.111331,111.133124,0.999,1924,10,3,1924.1,1924,1,3,1,0.009602
1924-10-04,1924-10-15,100.0,1.111331,111.133124,0.999,1924,10,4,1924.1,1924,1,4,1,0.009602
1924-10-05,1924-10-16,100.0,1.111331,111.133124,0.999,1924,10,5,1924.1,1924,1,5,1,0.009602
1924-10-06,1924-10-17,100.0,1.111331,111.133124,0.999,1924,10,6,1924.1,1924,1,6,1,0.009602
1924-10-07,1924-10-18,100.0,1.111331,111.133124,0.999,1924,10,7,1924.1,1924,1,7,1,0.009602
1924-10-08,1924-10-19,100.0,1.111331,111.133124,0.999,1924,10,8,1924.1,1924,1,8,2,0.009602


## Setup the Monthly Data

Load the monthly data.

In [115]:

monthly=daily.groupby(['MonthId','Year','Month']).size().to_frame(name="Days").reset_index(['Month','Year'])
monthly

,Year,Month,Days
MonthId,,,
1924.10,1924,10,31
1924.11,1924,11,30
1924.12,1924,12,31
1925.01,1925,1,31
1925.02,1925,2,28
...,...,...,...
2017.05,2017,5,31
2017.06,2017,6,30
2017.07,2017,7,31


Set the index and add additional columns for later use.

In [116]:

monthly['Day']=1
monthly['DateStart']=pd.to_datetime(monthly[['Year','Month','Day']])
monthly=monthly.drop('Day',1)


monthly['WaterYear']=monthly.apply((lambda x: (x['Year'] if x['Month']>=10 else x['Year']-1)),axis=1)
monthly['WaterMonth']=monthly.apply((lambda x: (x['Month']-9 if x['Month']>=10 else x['Month']+3)),axis=1)


monthly

,Year,Month,Days,DateStart,WaterYear,WaterMonth
MonthId,,,,,,
1924.10,1924,10,31,1924-10-01,1924,1
1924.11,1924,11,30,1924-11-01,1924,2
1924.12,1924,12,31,1924-12-01,1924,3
1925.01,1925,1,31,1925-01-01,1924,4
1925.02,1925,2,28,1925-02-01,1924,5
...,...,...,...,...,...,...
2017.05,2017,5,31,2017-05-01,2016,8
2017.06,2017,6,30,2017-06-01,2016,9
2017.07,2017,7,31,2017-07-01,2016,10


## Monthly flow summaries

Add flow summaries to the monthly data

In [117]:
monthly['Flow_min']=daily[['MonthId','Flow']].groupby('MonthId').min()
monthly['Flow_mean']=daily[['MonthId','Flow']].groupby('MonthId').mean()
monthly['Flow_median']=daily[['MonthId','Flow']].groupby('MonthId').median()
monthly['Flow_max']=daily[['MonthId','Flow']].groupby('MonthId').max()
monthly['Volume']=daily[['MonthId','Volume']].groupby('MonthId').sum()
monthly['Flow_range']=monthly['Flow_max']-monthly['Flow_min']
monthly[['Flow_min','Flow_mean','Flow_median','Flow_max','Flow_range']]
monthly

,Year,Month,Days,DateStart,WaterYear,WaterMonth,Flow_min,Flow_mean,Flow_median,Flow_max,Volume,Flow_range
MonthId,,,,,,,,,,,,
1924.10,1924,10,31,1924-10-01,1924,1,111.133124,116.046674,111.133124,126.236260,0.310819,15.103136
1924.11,1924,11,30,1924-11-01,1924,2,127.722788,153.527057,144.946329,191.196189,0.397942,63.473401
1924.12,1924,12,31,1924-12-01,1924,3,192.534369,275.311093,273.832492,438.500503,0.737393,245.966134
1925.01,1925,1,31,1925-01-01,1924,4,418.972890,691.530484,593.052584,1177.094832,1.852195,758.121942
1925.02,1925,2,28,1925-02-01,1924,5,1106.978245,1193.453064,1151.212624,1437.517109,2.887202,330.538864
...,...,...,...,...,...,...,...,...,...,...,...,...
2017.05,2017,5,31,2017-05-01,2016,8,1632.886017,2194.251076,2165.645561,2765.393133,5.877082,1132.507116
2017.06,2017,6,30,2017-06-01,2016,9,673.137147,1117.935234,1121.967762,1609.356985,2.897688,936.219838
2017.07,2017,7,31,2017-07-01,2016,10,381.684831,482.543823,464.672156,642.847453,1.292445,261.162622


## Annual Flow

Create a data table for annual (water year) summaries and populate.

In [118]:
yearly=monthly[['WaterYear']].groupby('WaterYear').count()

yearly['Flow_min']=daily[['WaterYear','Flow']].groupby('WaterYear').min()
yearly['Flow_median']=daily[['WaterYear','Flow']].groupby('WaterYear').median()
yearly['Flow_mean']=daily[['WaterYear','Flow']].groupby('WaterYear').mean()
yearly['Flow_max']=daily[['WaterYear','Flow']].groupby('WaterYear').max()
yearly['Flow_range']=yearly['Flow_max']-yearly['Flow_min']
yearly['Volume']=monthly[['WaterYear','Volume']].groupby('WaterYear').sum()

yearly

,Flow_min,Flow_median,Flow_mean,Flow_max,Flow_range,Volume
WaterYear,,,,,,
1924,111.133124,560.667874,991.317021,3456.928400,3345.795276,31.262174
1925,157.205464,497.238274,1111.041962,4504.008836,4346.803371,35.037819
1926,192.534369,560.667874,990.272631,3305.305419,3112.771050,31.229238
1927,178.800013,504.250658,847.079013,2256.674509,2077.874496,26.786671
1928,195.867340,390.023013,564.051561,1587.035405,1391.168064,17.787930
...,...,...,...,...,...,...
2012,224.058948,694.310091,1336.713717,3617.058785,3392.999837,42.154604
2013,194.855402,809.181755,1340.069290,3578.198961,3383.343558,42.260425
2014,205.608866,538.465854,696.864247,1460.437582,1254.828716,21.976311


In [119]:
Flow_mean_mean=yearly['Flow_mean'].describe()['mean']
Flow_max_mean=yearly['Flow_max'].describe()['mean']
Flow_min_mean=yearly['Flow_min'].describe()['mean']
Volume_mean=yearly['Volume'].describe()['mean']


yearly['Flow_mean_pct_var']=(yearly['Flow_mean']-Flow_mean_mean)/Flow_mean_mean*100
yearly['Flow_max_pct_var']=(yearly['Flow_max']-Flow_max_mean)/Flow_max_mean*100
yearly['Flow_min_pct_var']=(yearly['Flow_min']-Flow_min_mean)/Flow_min_mean*100
yearly['Volume_pct_var']=(yearly['Volume']-Volume_mean)/Volume_mean*100


Flow_mean_mean

1096.0061572902757

In [120]:
yearly['Flow_mean_5yr_mvCoefVar']=yearly['Flow_mean'].rolling(5,center=True).std()/Flow_mean_mean*100
yearly.loc[:,['Flow_mean_pct_var','Volume_pct_var']]

,Flow_mean_pct_var,Volume_pct_var
WaterYear,,
1924,-9.551875,-9.614174
1925,1.371872,1.302049
1926,-9.647165,-9.709399
1927,-22.712203,-22.553836
1928,-48.535731,-48.571178
...,...,...
2012,21.962245,21.878240
2013,22.268409,22.184193
2014,-36.417853,-36.461647


## Calendar months

Produce summaries of flow by calendar month

In [121]:
calmonthly=pd.DataFrame({'WaterMonth': [1,2,3,4,5,6,7,8,9,10,11,12],'MonthName': ['Oct','Nov','Dec','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep'], 'Month':[10,11,12,1,2,3,4,5,6,7,8,9]})
calmonthly=calmonthly.set_index('WaterMonth')
calmonthly['Flow_min']=daily[['WaterMonth','Flow']].groupby('WaterMonth').min()
calmonthly['Flow_mean']=daily[['WaterMonth','Flow']].groupby('WaterMonth').mean()
calmonthly['Flow_median']=daily[['WaterMonth','Flow']].groupby('WaterMonth').median()
calmonthly['Flow_max']=daily[['WaterMonth','Flow']].groupby('WaterMonth').max()
calmonthly['Flow_std']=daily[['WaterMonth','Flow']].groupby('WaterMonth').std()
calmonthly['Flow_coefvar']=(calmonthly['Flow_std']/calmonthly['Flow_mean']*100).round(1)

calmonthly

,MonthName,Month,Flow_min,Flow_mean,Flow_median,Flow_max,Flow_std,Flow_coefvar
WaterMonth,,,,,,,,
1,Oct,10,104.977648,238.595579,230.611546,471.306385,60.224640,25.2
2,Nov,11,102.744081,275.881399,263.127095,641.262334,80.709314,29.3
3,Dec,12,186.876438,431.992745,408.384252,1127.620860,137.994172,31.9
4,Jan,1,273.832492,718.116610,629.767856,3888.545559,346.143364,48.2
5,Feb,2,390.023013,1346.147442,963.050617,9911.758819,1164.527759,86.5
6,Mar,3,504.250658,2529.652693,2172.233613,9529.570904,1656.181136,65.5
7,Apr,4,685.792889,2964.457896,2962.514654,8542.719391,1354.973279,45.7
8,May,5,421.711275,2218.080459,2157.694543,5673.759847,924.785719,41.7
9,Jun,6,273.832492,1201.478405,1127.620860,3926.960162,550.803730,45.8


## Calendar Month Flow exceedance

Flow exceedance values by calendar month.

P90 is flow which is exceeded for 90% of the time.

In [122]:

calmonthly['Flow_P95']=monthly[['WaterMonth','Flow_mean']].groupby('WaterMonth').quantile(0.05)
calmonthly['Flow_P90']=monthly[['WaterMonth','Flow_mean']].groupby('WaterMonth').quantile(0.1)
#calmonthly['Flow_P80']=monthly[['Month','Flow_mean']].groupby('Month').quantile(0.2)
calmonthly['Flow_P75']=monthly[['WaterMonth','Flow_mean']].groupby('WaterMonth').quantile(0.25)
calmonthly['Flow_P50']=monthly[['WaterMonth','Flow_mean']].groupby('WaterMonth').quantile(0.5)
calmonthly['Flow_P25']=monthly[['WaterMonth','Flow_mean']].groupby('WaterMonth').quantile(0.75)
#calmonthly['Flow_P20']=monthly[['Month','Flow_mean']].groupby('Month').quantile(0.8)
calmonthly['Flow_P10']=monthly[['WaterMonth','Flow_mean']].groupby('WaterMonth').quantile(0.9)
calmonthly['Flow_P05']=monthly[['WaterMonth','Flow_mean']].groupby('WaterMonth').quantile(0.95)

calmonthly

,MonthName,Month,Flow_min,Flow_mean,Flow_median,Flow_max,Flow_std,Flow_coefvar,Flow_P95,Flow_P90,Flow_P75,Flow_P50,Flow_P25,Flow_P10,Flow_P05
WaterMonth,,,,,,,,,,,,,,,
1,Oct,10,104.977648,238.595579,230.611546,471.306385,60.224640,25.2,163.960680,172.337628,199.372091,228.637711,274.461350,308.102591,343.273895
2,Nov,11,102.744081,275.881399,263.127095,641.262334,80.709314,29.3,176.031597,199.755714,221.011317,266.323478,309.866716,375.815427,409.593174
3,Dec,12,186.876438,431.992745,408.384252,1127.620860,137.994172,31.9,289.626432,308.051521,346.412998,402.301251,503.699189,562.685065,685.355162
4,Jan,1,273.832492,718.116610,629.767856,3888.545559,346.143364,48.2,436.280035,478.628811,542.101989,620.129230,811.344208,996.195907,1314.113257
5,Feb,2,390.023013,1346.147442,963.050617,9911.758819,1164.527759,86.5,626.409663,690.839238,807.233391,986.633624,1417.444938,2478.096386,3201.226914
6,Mar,3,504.250658,2529.652693,2172.233613,9529.570904,1656.181136,65.5,848.541223,909.825644,1184.390848,2339.808237,3120.855022,4345.984890,5638.829815
7,Apr,4,685.792889,2964.457896,2962.514654,8542.719391,1354.973279,45.7,1085.442626,1256.028764,1922.361846,2960.216858,3699.347853,4654.522435,5104.693213
8,May,5,421.711275,2218.080459,2157.694543,5673.759847,924.785719,41.7,876.335485,1128.544360,1682.373622,2234.561289,2673.535307,3215.914788,3623.112951
9,Jun,6,273.832492,1201.478405,1127.620860,3926.960162,550.803730,45.8,450.280286,631.415271,853.629555,1176.988450,1466.071086,1810.876136,1972.049873


## Prepare the Representative Years Summaries

In [123]:
selected = pd.read_csv(input_data + "selected_years.csv").rename(columns={"Year": "WaterYear"}).set_index('WaterYear')
selected['Flow_min']=yearly['Flow_min']
selected['Flow_mean']=yearly['Flow_mean']
selected['Flow_max']=yearly['Flow_max']
selected['Volume']=yearly['Volume']

selected

,Class,Flow_Exceedance,Flow_min,Flow_mean,Flow_max,Volume
WaterYear,,,,,,
1967,Very Wet,Q3,254.145972,1844.708478,5511.046404,58.334109
2013,Wet,Q12,194.855402,1340.069290,3578.198961,42.260425
2002,Median,Q50,194.855402,1072.634961,3847.378538,33.826616
1990,Dry,Q90,197.600309,770.503828,2291.801629,24.298609
1996,Very Dry,Q97,102.744081,546.340570,1398.388034,17.229396


## Save the Data

In [124]:
daily.to_csv(output_data + 'ngonye_flow_daily.csv')
monthly.to_csv(output_data + 'ngonye_flow_monthly.csv')
yearly.to_csv(output_data + 'ngonye_flow_yearly.csv')
calmonthly.to_csv(output_data + 'ngonye_flow_calmonthly.csv')
selected.to_csv(output_data + 'ngonye_flow_selected_years.csv')